In [1]:
import pandas as pd
import csv
from datetime import datetime, timedelta
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
input_file="/content/drive/MyDrive/input.csv"

In [4]:
df = pd.read_csv(input_file)

In [ ]:
df.head()


In [ ]:
# Check for missing values
print("Missing values in each column:")
print(df.isnull().sum())

# Investigate missing values in the 'Compensation' column
print("\nSummary statistics for Compensation column:")
print(df['Compensation'].describe())



In [11]:
# Convert date columns to datetime objects
date_columns = ['Date of Joining', 'Date of Exit', 'Compensation 1 date', 'Compensation 2 date',
                'Review 1 date', 'Review 2 date', 'Engagement 1 date', 'Engagement 2 date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

# Function to derive effective and end dates
def derive_dates(row):
    if pd.isnull(row['Date of Exit']):
        end_date = datetime(2100, 1, 1)
    else:
        end_date = row['Date of Exit'] - timedelta(days=1)
    return pd.Series([row['Date of Joining'], end_date])

# Apply function to derive dates
df[['Effective Date', 'End Date']] = df.apply(derive_dates, axis=1)

# Fill missing values with the most recent past record for the same employee
df = df.ffill()

# Create list to hold records
records = []

# Iterate over each row to generate historical records
for index, row in df.iterrows():
    last_compensation = None
    last_pay_raise_date = None

    # Generate quarterly intervals between 'Effective Date' and 'End Date'
    quarter_intervals = pd.date_range(start=row['Effective Date'], end=row['End Date'], freq='Q')

    # Iterate over each quarterly interval
    for i in range(len(quarter_intervals) - 1):
        start_date = quarter_intervals[i]
        end_date = quarter_intervals[i + 1] - timedelta(days=1)

        # Generate a historical record for each quarterly interval
        record = {
            'Employee Code': row['Employee Code'],
            'Manager Employee Code': row['Manager Employee Code'],
            'Last Compensation': last_compensation,
            'Compensation': row['Compensation'],
            'Last Pay Raise Date': last_pay_raise_date,
            'Variable Pay': row['Compensation 1'],
            'Tenure in Org': (end_date - row['Date of Joining']).days,
            'Performance Rating': row['Review 2'],
            'Engagement Score': row['Engagement 2'],
            'Effective Date': start_date,
            'End Date': end_date
        }

        # Update last compensation and last pay raise date for the next iteration
        last_compensation = row['Compensation']
        last_pay_raise_date = row['Compensation 2 date']

        # Append the record to the list
        records.append(record)

# Create output DataFrame by concatenating records
output_df = pd.DataFrame(records)

# Write output to CSV file
output_df.to_csv("output_file.csv", index=False)